In [24]:
import pandas as pd
import shapefile
import numpy as np
from shapely.geometry import Point, Polygon
from math import radians, cos, sin, asin, sqrt
import json 
import os

In [25]:
house_df = pd.read_csv('./dataset/house_listing_complete_2022.csv')
house_df.head(3)

,Unnamed: 0,sold_price,list_price,address,web_link,bedroom,bathroom,sqft,link,type,level,parking,level_0,listing_gmap_urls,lat,long
0,0,"$1,680,000","List Price$1,499,000",30 Norway Ave,/toronto-on-real-estate/30-norway-ave,3 Bed,3 Bath,N/A sqft,/toronto-on-real-estate/30-norway-ave,TypeSemi-Detached,Levels2-Storey,1.0,0,https://www.google.com/maps/place/30+Norway+Av...,43.672653,-79.314617
1,1,"$600,000","List Price$550,000",1403 - 76 Shuter St,/toronto-on-real-estate/1403-76-shuter-st,1 Bed,1 Bath,0-499 sqft,/toronto-on-real-estate/1403-76-shuter-st,TypeCondo Apt,LevelsApartment,1.0,1,https://www.google.com/maps/place/76+Shuter+St...,43.655293,-79.384117
2,2,"$660,000","List Price$599,000",103 - 80 Marine Parade Dr,/toronto-on-real-estate/103-80-marine-parade-dr,1 Bed,1 Bath,500â€“599 sqft,/toronto-on-real-estate/103-80-marine-parade-dr,TypeCondo Apt,LevelsApartment,1.0,2,https://www.google.com/maps/place/80+Marine+Pa...,43.626188,-79.486044


In [26]:
subwaydf = pd.read_excel('./dataset/TTC_Stn.xlsx')
subwaydf.head(3)

,Unnamed: 0,STATION,LINE,PLATFORM_L,AVG_PASSEN,LINE2,PLATFORM_1,SUBWAY_TRA,ADDRESS,Opened,geometry,lat,long
0,0,Kipling,Bloor-Danforth,1,53640,NaN,NaN,False,5247 Dundas St. West,1980,POINT (618101.6129999999 4832636.300000001),43.637599,-79.535530
1,1,Islington,Bloor-Danforth,1,43090,NaN,NaN,False,3286 Bloor St. West,1968,POINT (618990.6128000002 4833544.113399999),43.645027,-79.524574
2,2,Royal York,Bloor-Danforth,2,19440,NaN,NaN,False,3012 Bloor St. West,1968,POINT (620056.4956999999 4833882.763800001),43.648220,-79.511319


In [27]:
def dist_subway(list_loc, df):
    s  = []
    for i in range(len(df)):
        lon1 = radians(list_loc[1])
        lon2 = radians(df.loc[i, "long"])
        lat1 = radians(list_loc[0])
        lat2 = radians(df.loc[i, "lat"])
        
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        dist = 2 * asin(sqrt(a))*6371
        s.append([df.loc[i, "LINE"], df.loc[i,"STATION"], dist])
    s.sort(key=lambda z: z[2])
    return s[0]

subway_min = []
for ri in range(len(house_df)):
    list_loc = [house_df.loc[ri, "lat"], house_df.loc[ri,"long"]]
    subway_min.append(dist_subway(list_loc, subwaydf))

In [28]:
print(subway_min)

[['Bloor-Danforth', 'Coxwell', 1.4483041284552154], ['Yonge-University-Spadina', 'Dundas', 0.31412180253552957], ['Bloor-Danforth', 'Jane', 2.6555485005611517], ['Sheppard', 'Bessarion', 0.39899853352896125], ['Yonge-University-Spadina', "Queen's Park", 0.34990436075735315], ['Bloor-Danforth', 'Kennedy', 3.2360453697296574], ['Bloor-Danforth', 'High Park', 0.8843482035475743], ['Bloor-Danforth', 'Main Street', 0.7674225911360094], ['Sheppard', 'Don Mills', 1.6005967951246607], ['Yonge-University-Spadina', 'Osgoode', 1.1984818476842698], ['Bloor-Danforth', 'Dufferin', 1.993579610457049], ['Yonge-University-Spadina', 'Union', 0.5570553872730918], ['Bloor-Danforth', 'Castle Frank', 1.2650196149014397], ['Yonge-University-Spadina', "Queen's Park", 0.3455631006790533], ['Bloor-Danforth', 'Kennedy', 5.525945577362392], ['Sheppard', 'Bessarion', 0.34938147837801664], ['Yonge-University-Spadina', 'Glencairn', 1.5358337815236878], ['Bloor-Danforth', 'Bay', 0.3186397310907314], ['Bloor-Danforth'

In [31]:
subway_mindf = pd.DataFrame(subway_min, columns=["Line", "Station", "Dist"])
subway_mindf.head(3)

,Line,Station,Dist
0,Bloor-Danforth,Coxwell,1.448304
1,Yonge-University-Spadina,Dundas,0.314122
2,Bloor-Danforth,Jane,2.655549


In [32]:
listing_subwaydf = pd.concat([house_df,subway_mindf], axis=1)
listing_subwaydf.head(10)

,Unnamed: 0,sold_price,list_price,address,web_link,bedroom,bathroom,sqft,link,type,level,parking,level_0,listing_gmap_urls,lat,long,Line,Station,Dist
0,0,"$1,680,000","List Price$1,499,000",30 Norway Ave,/toronto-on-real-estate/30-norway-ave,3 Bed,3 Bath,N/A sqft,/toronto-on-real-estate/30-norway-ave,TypeSemi-Detached,Levels2-Storey,1.0,0,https://www.google.com/maps/place/30+Norway+Av...,43.672653,-79.314617,Bloor-Danforth,Coxwell,1.448304
1,1,"$600,000","List Price$550,000",1403 - 76 Shuter St,/toronto-on-real-estate/1403-76-shuter-st,1 Bed,1 Bath,0-499 sqft,/toronto-on-real-estate/1403-76-shuter-st,TypeCondo Apt,LevelsApartment,1.0,1,https://www.google.com/maps/place/76+Shuter+St...,43.655293,-79.384117,Yonge-University-Spadina,Dundas,0.314122
2,2,"$660,000","List Price$599,000",103 - 80 Marine Parade Dr,/toronto-on-real-estate/103-80-marine-parade-dr,1 Bed,1 Bath,500â€“599 sqft,/toronto-on-real-estate/103-80-marine-parade-dr,TypeCondo Apt,LevelsApartment,1.0,2,https://www.google.com/maps/place/80+Marine+Pa...,43.626188,-79.486044,Bloor-Danforth,Jane,2.655549
3,3,"$825,000","List Price$699,000",2105 - 115 Mcmahon Dr,/toronto-on-real-estate/2105-115-mcmahon-dr,2 Bed,1 Bath,700â€“799 sqft,/toronto-on-real-estate/2105-115-mcmahon-dr,TypeCondo Apt,LevelsApartment,1.0,3,https://www.google.com/maps/place/115+McMahon+...,43.766876,-79.379817,Sheppard,Bessarion,0.398999
4,4,"$910,000","List Price$869,000",2203 - 750 Bay St,/toronto-on-real-estate/2203-750-bay-st,2 Bed,2 Bath,800â€“899 sqft,/toronto-on-real-estate/2203-750-bay-st,TypeCondo Apt,LevelsApartment,1.0,4,"https://www.google.com/maps/place/750+Bay+St.,...",43.659682,-79.394818,Yonge-University-Spadina,Queen's Park,0.349904
5,5,"$1,501,000","List Price$999,000",50 Portsdown Rd,/toronto-on-real-estate/50-portsdown-rd,3 Bed,4 Bath,N/A sqft,/toronto-on-real-estate/50-portsdown-rd,TypeDetached,LevelsBungalow,4.0,5,https://www.google.com/maps/place/50+Portsdown...,43.753546,-79.291461,Bloor-Danforth,Kennedy,3.236045
6,6,"$1,660,000","List Price$1,099,000",77 Medland Cres,/toronto-on-real-estate/77-medland-cres,3 Bed,2 Bath,N/A sqft,/toronto-on-real-estate/77-medland-cres,TypeSemi-Detached,Levels2-Storey,4.0,6,https://www.google.com/maps/place/77+Medland+C...,43.660289,-79.473264,Bloor-Danforth,High Park,0.884348
7,7,"$1,475,000","List Price$1,099,000",377 Scarborough Rd,/toronto-on-real-estate/377-scarborough-rd,3 Bed,2 Bath,N/A sqft,/toronto-on-real-estate/377-scarborough-rd,TypeSemi-Detached,Levels2-Storey,0.0,7,https://www.google.com/maps/place/377+Scarboro...,43.683355,-79.296773,Bloor-Danforth,Main Street,0.767423
8,8,"$650,000","List Price$499,000",801 - 3121 Sheppard Ave E,/toronto-on-real-estate/801-3121-sheppard-ave-e,1 Bed,1 Bath,600â€“699 sqft,/toronto-on-real-estate/801-3121-sheppard-ave-e,TypeCondo Apt,LevelsApartment,1.0,8,https://www.google.com/maps/place/3121+Sheppar...,43.776256,-79.325440,Sheppard,Don Mills,1.600597
9,9,"$720,000","List Price$578,800",2310 - 125 Peter St,/toronto-on-real-estate/2310-125-peter-st,1 Bed,1 Bath,0-499 sqft,/toronto-on-real-estate/2310-125-peter-st,TypeCondo Apt,LevelsApartment,0.0,9,https://www.google.com/maps/place/125+Peter+St...,43.648475,-79.401431,Yonge-University-Spadina,Osgoode,1.198482


In [23]:
def find_district(point, df):
    in_shape = []
    for sh in df.geometry:
        within = point.within(sh)
        in_shape.append(within)
    return df.loc[in_shape.index(True), 'area'], df.loc[in_shape.index(True), 'area_code'], df.loc[in_shape.index(True), 'average_income']


houses['mean_district_income'] = 0
houses['district_code'] = 0

In [ ]:
listing_subwaydf.to_csv("house_complete_ttc_2022.csv")